In [1]:
# Install the ultralytics package from PyPI
# !pip install ultralytics

In [2]:
# import cv2

# from ultralytics import YOLO

# # Load the YOLO model
# model = YOLO("yolo11n.pt")

# # Open the video file
# video_path = "/videos/my_video.mp4"
# cap = cv2.VideoCapture(video_path)

# print(cap)

# # Loop through the video frames
# while cap.isOpened():
#     # Read a frame from the video
#     success, frame = cap.read()

#     if success:
#         # Run YOLO inference on the frame
#         results = model(frame)
#         print("Successful")

#         # Visualize the results on the frame
#         annotated_frame = results[0].plot()

#         # Display the annotated frame
#         cv2.imshow("YOLO Inference", annotated_frame)

#         # Break the loop if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord("q"):
#             break
#     else:
#         # Break the loop if the end of the video is reached
#         print("Not Successful")

#         break

# # Release the video capture object and close the display window
# cap.release()
# cv2.destroyAllWindows()

**Importations of Packages / Modules**

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
# from tf_keras.models import load_model
from ultralytics import YOLO
import torch







 **Data Preparation**

In [17]:
# Load images from the directory
# directory_path = "/content/drive/MyDrive/Digital Engineering/engg680_2024_fall/Semester_Project/image_directory"
# def load_images_from_directory(directory_path):
#     images = []
#     for filename in os.listdir(directory_path):
#         img = cv2.imread(os.path.join(directory_path, filename))
#         if img is not None:
#             images.append(img)
#     print(images)
#     return images

# Preprocess data
def preprocess_image(image, input_size=(416, 416)):
    image_resized = cv2.resize(image, input_size)
    image_normalized = image_resized / 255.0
    return np.expand_dims(image_normalized, axis=0)


In [ ]:
image_list = load_images_from_directory("")
# preprocess_image
processed_images = []
for image in image_list:
  processed_images.append(preprocess_image(image))

print(processed_images)

FileNotFoundError: [Errno 2] No such file or directory: ''

**Model Definition**

In [18]:

def load_yolo_model(model_path):
    model = torch.load(model_path)
    return model


**Definition of Detection Functions**

In [27]:
# Non-Maximum Suppression (NMS)
def non_max_suppression(predictions, confidence_threshold, nms_threshold):
    boxes, confidences, class_ids = [], [], []

    for prediction in predictions:
        confidence = prediction[4]
        if confidence > confidence_threshold:
            x, y, w, h = prediction[:4]
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(np.argmax(prediction[5:]))

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    # nms_boxes = [(boxes[i], confidences[i], class_ids[i]) for i in indices.flatten()]
    nms_boxes = [(boxes[i], confidences[i], class_ids[i]) for i in indices]
    # nms_boxes = [(boxes[i[0]], confidences[i[0]], class_ids[i[0]]) for i in indices]

    return nms_boxes

# Bounding box decoding
def decode_bounding_boxes(predictions, input_shape=(416, 416)):
    boxes = []
    for pred in predictions:
        x, y, w, h = pred[:4]
        box = [int(x * input_shape[0]), int(y * input_shape[1]),
               int(w * input_shape[0]), int(h * input_shape[1])]
        boxes.append(box)
    return boxes

# Prediction and Post-processing
# def detect_objects(model, image, confidence_threshold=0.5, nms_threshold=0.4):
#     input_data = preprocess_image(image)
#     predictions = model.predict(input_data)
#     boxes = decode_bounding_boxes(predictions[0])
#     results = non_max_suppression(predictions[0], confidence_threshold, nms_threshold)
#     return results

def detect_objects(model, image, confidence_threshold=0.5, nms_threshold=0.4):
    # Preprocess the image as needed by your model
    input_data = preprocess_image(image)
    
    # Ensure the model is in evaluation mode
    # model.eval()
    
    # Run inference (forward pass) through the model
    with torch.no_grad():  # Disable gradient computation for inference
        predictions = model(input_data)  # Directly use model(input_data) instead of model.predict(input_data)
    
    # Process the predictions
    boxes = decode_bounding_boxes(predictions[0])
    results = non_max_suppression(predictions[0], confidence_threshold, nms_threshold)
    
    return results



**Run Object Detection on New Data**

In [21]:
# Draw Bounding Boxes

def draw_bounding_boxes(frame, boxes, confidences, class_ids, classes):
    for (box, confidence, class_id) in zip(boxes, confidences, class_ids):
        x, y, w, h = box
        label = str(classes[class_id])
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    return frame


Algorithm for Trainig Model From Scratch

In [ ]:
# Compile model for training
# def compile_model(model):
#     model.compile(optimizer='adam', loss='categorical_crossentropy')

# # Training Loop
# def train_yolo(model, dataset, epochs):
#     for epoch in range(epochs):
#         for images, labels in dataset:
#             with tf.GradientTape() as tape:
#                 predictions = model(images)
#                 loss = tf.reduce_mean(tf.losses.categorical_crossentropy(labels, predictions))
#             gradients = tape.gradient(loss, model.trainable_variables)
#             model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#             print(f"Epoch {epoch+1}, Loss: {loss.numpy()}")
#     model.save('trained_yolo_model.h5')


**Evaluation and Testing**

In [28]:
# Evaluate model
# def evaluate_model(model, test_dataset):
#     total_iou, count = 0, 0
#     for images, true_boxes in test_dataset:
#         predictions = model(images)
#         pred_boxes = decode_bounding_boxes(predictions)
#         iou = calculate_iou(pred_boxes, true_boxes)  # Assuming a helper IoU function
#         total_iou += iou
#         count += 1
#     print(f"Mean IoU: {total_iou / count}")

# Real-time Detection from Webcam or Video
def detect_in_realtime(model, video_source=0):
    cap = cv2.VideoCapture(video_source)
    print(cap)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = detect_objects(model, frame)
        frame = draw_bounding_boxes(frame, *zip(*results))
        cv2.imshow('YOLO Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

model_path = "yolo11n.pt"
model = torch.load(model_path)
# model.eval()  #
# compile_model(model)
# train_yolo(model, dataset, epochs=10)
# evaluate_model(model, test_dataset
detect_in_realtime(model)


< cv2.VideoCapture 00000140B2F45890>


TypeError: 'dict' object is not callable